__качаем объявления о продаже земельных участков с авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
place = 'sevastopol'
avito_path = place+'/zemelnye_uchastki/prodam/izhs'
    
raw_data_path = 'data/raw/land/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-09-19 12:38:09 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-19 12:38:09 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-19 12:38:11 | AdsListParser: downloader init
[INFO    ] 2022-09-19 12:38:11 | AvitoParser: init
[INFO    ] 2022-09-19 12:38:20 | AvitoParser: 23 pages for read
[INFO    ] 2022-09-19 12:38:20 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-19 12:38:23 | AdsListParser: read page 1
[INFO    ] 2022-09-19 12:38:28 | AdsListParser: read page 2
[INFO    ] 2022-09-19 12:38:31 | AdsListParser: read page 3
[INFO    ] 2022-09-19 12:38:35 | AdsListParser: read page 4
[INFO    ] 2022-09-19 12:38:39 | AdsListParser: read page 5
[INFO    ] 2022-09-19 12:38:45 | AdsListParser: read page 6
[INFO    ] 2022-09-19 12:38:49 | AdsListParser: read page 7
[INFO    ] 2022-09-19 12:38:53 | AdsListParser: read page 8
[INFO    ] 2022-09-19 12:38:58 | AdsListParser: read page 9
[INFO    ] 2022-09-19 12:39:04 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

1104
1104


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [10]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts,place
437,2448235138,Объявление «Участок 8 сот. (ИЖС)» 5 фотографий,3500000,,"с. Флотское, Ягодная ул.","Арт. 24777666 Торг после осмотра. \n\nПродам видовой участок 8 соток (Ижс), село Флотское. \nКоммуникации: \nэлектричество — 15 квт.\nВода — центральный водопровод по границе участка.\n\nГаз — проведение запланировано на конец 2022 года.\n\nУчасток правильной формы",9,2022-09-19 12:39:53.571271,sevastopol
411,2529306780,"Объявление «Участок 7,5 сот. (ИЖС)» с фотографией",7000000,,"Пасечный пр., 3","Участок 7,5 соток ижс. Выход на 2 улицы. На участке проведено вода, свет ( заведено 380), очистные ( сданы в архитектуру), видеонаблюдение ( +онлайн), интернет. Имеется разрешение на строительство дома все документы чистые, российские. Один собственник Я.",9,2022-09-19 12:39:53.571271,sevastopol
